## Importing libraries

In [33]:
# pip install opencv-python pillow bs4


In [1]:
import tensorflow
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from tensorflow.keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
import os
import json
import requests
from bs4 import BeautifulSoup
from PIL import Image
import re
import smtplib
from email.mime.text import MIMEText
# import wget
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from requests.exceptions import MissingSchema, InvalidURL

# Web Extraction

In [2]:
source_url = "https://sites.google.com/site/pornographydatabase/"
response = requests.get(source_url)
soup = BeautifulSoup(response.text, "html.parser")

image_urls = []
for img in soup.find_all("img"):
    image_url = img.get("src")
    if image_url:
        image_urls.append(image_url)

print(image_urls)

['https://sites.google.com/site/pornographydatabase/_/rsrc/1468849941795/home/pornDB2.png']


In [3]:
# filename = wget.download('https://figshare.com/ndownloader/files/27771210')

# Model Training - Classification



In [4]:
# !wget https://figshare.com/ndownloader/files/27771210

In [5]:
# !unzip /content/27771210

In [6]:
# Define the image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('Imageandvideodataset/image dataset/train', target_size=(299, 299), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory('Imageandvideodataset/image dataset/test', target_size=(299, 299), batch_size=32, class_mode='binary')

Found 22306 images belonging to 2 classes.
Found 8881 images belonging to 2 classes.


In [7]:
# Load the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer with 1024 units and ReLU activation
x = Dense(1024, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
#x = Dense(256, activation='relu')(x)
# x = Dense(64, activation='relu')(x)

# Add a final output layer with sigmoid activation for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, steps_per_epoch=10, validation_data=test_generator, validation_steps=10)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=10)
print('Test accuracy:', test_acc)

Epoch 1/10
10/10 [==============================] - 260s 23s/step - loss: 0.4723 - accuracy: 0.7812 - val_loss: 0.5230 - val_accuracy: 0.7937
Epoch 2/10
10/10 [==============================] - 222s 22s/step - loss: 0.1946 - accuracy: 0.9344 - val_loss: 0.6890 - val_accuracy: 0.8156
Epoch 3/10
10/10 [==============================] - 215s 22s/step - loss: 0.2472 - accuracy: 0.9125 - val_loss: 0.4979 - val_accuracy: 0.8500
Epoch 4/10
10/10 [==============================] - 419s 44s/step - loss: 0.2352 - accuracy: 0.9156 - val_loss: 0.7183 - val_accuracy: 0.8000
Epoch 5/10
10/10 [==============================] - 214s 21s/step - loss: 0.2051 - accuracy: 0.9156 - val_loss: 0.3920 - val_accuracy: 0.8875
Epoch 6/10
10/10 [==============================] - 219s 22s/step - loss: 0.1698 - accuracy: 0.9344 - val_loss: 0.5323 - val_accuracy: 0.8500
Epoch 7/10
10/10 [==============================] - 211s 21s/step - loss: 0.1729 - accuracy: 0.9438 - val_loss: 0.6565 - val_accuracy: 0.8375
Epoch 

In [9]:
model.save('model_94.h5')

In [35]:
pip install joblib

     ------------------------------------ 298.0/298.0 kB 107.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
from joblib import dump
dump('model_94.h5', 'savedmodels/model.joblib')

['savedmodels/model.joblib']

# Image prediction

In [10]:
im = Image.open(requests.get('https://sites.google.com/site/pornographydatabase/_/rsrc/1468849941795/home/pornDB2.png', stream=True).raw)

In [11]:
# Define a function to preprocess the input image
def preprocess_image(image):
    # Load the image
    #image = cv2.imread(image_path)
    # Resize the image to 224x224 pixels
    image = np.array(image)
    image = cv2.resize(image, (224, 224))
    # Convert the image to a 4D tensor
    image = np.expand_dims(image, axis=0)
    # Subtract the mean RGB values of the ImageNet dataset
    image = image - [123.68, 116.779, 103.939]
    return image

In [12]:
img = preprocess_image(im)

# Use the ResNet50 model to make a prediction
resnet_prediction = model.predict(img)

# Print the predictions
print('ResNet50 prediction:', resnet_prediction)

1/1 [==============================] - 2s 2s/step
ResNet50 prediction: [[4.4948e-13]]


# Video prediction

In [17]:
# set video file path of input video with name and extension
vid = cv2.VideoCapture('Imageandvideodataset/videos/v1.mp4')
# print(vid)

if not os.path.exists('images'):
    os.makedirs('images')

#for frame identity
index = 0


def getFrame(sec):
    vid.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vid.read()
    if hasFrames:
        cv2.imwrite("./images/frame"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 0.5 #//it will capture image in each 0.5 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)




# while(True):
#     # print(".")
#     # Extract images
#     ret, frame = vid.read()
    
#     # end of frames
#     if not ret: 
#         break
#     # Saves images
#     name = './images/frame' + str(index) + '.jpg'
#     print ('Creating...' + name)
#     cv2.imwrite(name, frame)

#     # next frame
#     index += 1

In [18]:
dir_path = r'images'
# print(dir_path)
res=[]

for path in os.listdir(dir_path):
    # print(path)
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(dir_path+'/'+path)
print(res)

['images/frame1.jpg', 'images/frame10.jpg', 'images/frame100.jpg', 'images/frame101.jpg', 'images/frame102.jpg', 'images/frame103.jpg', 'images/frame104.jpg', 'images/frame105.jpg', 'images/frame106.jpg', 'images/frame107.jpg', 'images/frame108.jpg', 'images/frame109.jpg', 'images/frame11.jpg', 'images/frame110.jpg', 'images/frame111.jpg', 'images/frame112.jpg', 'images/frame113.jpg', 'images/frame114.jpg', 'images/frame115.jpg', 'images/frame116.jpg', 'images/frame117.jpg', 'images/frame118.jpg', 'images/frame119.jpg', 'images/frame12.jpg', 'images/frame120.jpg', 'images/frame121.jpg', 'images/frame122.jpg', 'images/frame123.jpg', 'images/frame124.jpg', 'images/frame125.jpg', 'images/frame126.jpg', 'images/frame127.jpg', 'images/frame128.jpg', 'images/frame129.jpg', 'images/frame13.jpg', 'images/frame130.jpg', 'images/frame131.jpg', 'images/frame132.jpg', 'images/frame133.jpg', 'images/frame134.jpg', 'images/frame135.jpg', 'images/frame136.jpg', 'images/frame137.jpg', 'images/frame138

In [28]:
arr_obscene = []
cnt=0
for i in res:
  img = preprocess_image(cv2.imread(i))

  # print(img)
# Use the ResNet50 model to make a prediction
  resnet_prediction = model.predict(img)
  
  if(resnet_prediction<=10**-15 and resnet_prediction!=0.):
      cnt+=1
      arr_obscene.append(i)
  
# Print the predictions
  print('Prediction:', resnet_prediction)
print(arr_obscene)

1/1 [==============================] - 0s 186ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 184ms/step
Prediction: [[2.5673603e-16]]
1/1 [==============================] - 0s 189ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 188ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 159ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 144ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 115ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 120ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 115ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 109ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 115ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 114ms/step
Prediction: [[0.]]
1/1 [==============================] - 0s 112ms/step
Prediction: [[0.9999725]]
1/1 [==============================] - 0s 122m

In [29]:
if(cnt>100):
        print("Video contains obscene data")

Video contains obscene data


## FINAL MODELS

In [40]:
import json

with open('output.json') as f:
    data = json.load(f)

print(data)


{'url': 'https://divik2510.github.io/KavachProject', 'email': 'divikagrawal2510@gmail.com'}


### 1. DATA COLLECTION

In [42]:
#1. Data Collection

# CREATE FOLDER
def folder_create(images):
	try:
		folder_name = 'sample'
		# folder creation
		os.mkdir(folder_name)

	# if folder exists with that name, ask another name
	except:
		print("Folder Exist with that name!")
		folder_create()

	# image downloading start
	download_images(images, folder_name)


# DOWNLOAD ALL IMAGES FROM THAT URL
def download_images(images, folder_name):

	# initial count is zero
	count = 0

	# print total images found in URL
	print(f"Total {len(images)} Image Found!")

	# checking if images is not zero
	if len(images) != 0:
		for i, image in enumerate(images):
			# From image tag ,Fetch image Source URL

						# 1.data-srcset
						# 2.data-src
						# 3.data-fallback-src
						# 4.src

			# Here we will use exception handling

			# first we will search for "data-srcset" in img tag
			try:
				# In image tag ,searching for "data-srcset"
				image_link = image["data-srcset"]
				
			# then we will search for "data-src" in img
			# tag and so on..
			except:
				try:
					# In image tag ,searching for "data-src"
					image_link = image["data-src"]
				except:
					try:
						# In image tag ,searching for "data-fallback-src"
						image_link = image["data-fallback-src"]
					except:
						try:
							# In image tag ,searching for "src"
							image_link = image["src"]

						# if no Source URL found
						except:
							pass

			# After getting Image Source URL
			# We will try to get the content of image
			try:
				r = requests.get(image_link).content
				try:

					# possibility of decode
					r = str(r, 'utf-8')

				except UnicodeDecodeError:

					# After checking above condition, Image Download start
					with open(f"{folder_name}/images{i+1}.jpg", "wb+") as f:
						f.write(r)

					# counting number of image downloaded
					count += 1
			except:
				pass

		# There might be possible, that all
		# images not download
		# if all images download
		if count == len(images):
			print("All Images Downloaded!")
			
		# if all images not download
		else:
			print(f"Total {count} Images Downloaded Out of {len(images)}")

# MAIN FUNCTION START
def main(url):

	# content of URL
	r = requests.get(url)

	# Parse HTML Code
	soup = BeautifulSoup(r.text, 'html.parser')

	# find all images in URL
	images = soup.findAll('img')

	# Call folder create function
	folder_create(images)


# take url
url = data['url']

# CALL MAIN FUNCTION
main(url)

Total 10 Image Found!
All Images Downloaded!


### 2. UPSAMPLING

In [22]:
#2. Upsampling

path_cs  = 'sample'
# path_gi  = 'D:\CODING\Python programs\VITISH 23\content\gore-images'
# path_li  = 'D:\CODING\Python programs\VITISH 23\content\lingerie'
# path_nu  = 'D:\CODING\Python programs\VITISH 23\content\nudity'
# path_sa  = 'D:\CODING\Python programs\VITISH 23\content\sexual activity'

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# Loop through images and generate augmented versions
def upsample(image_path):
    for filename in os.listdir(image_path):
        img = load_img(os.path.join(image_path, filename))  # Load image
        x = img_to_array(img)  # Convert to numpy array
        x = x.reshape((1,) + x.shape)  # Reshape to 4D array with single sample
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=image_path, save_prefix=filename.split('.')[0], save_format='jpg'):
            i += 1
            if i > 9:  # Generate 10 augmented versions of each image
                break

upsample(path_cs)
# upsample(path_gi)
# upsample(path_li)
# upsample(path_nu)
# upsample(path_sa)

### 3. WEB SCRAPING

In [14]:
#3. Web scraping

def web_scrap(web_url):
  source_url = web_url
  response = requests.get(source_url)
  soup = BeautifulSoup(response.text, "html.parser")

  image_urls = []
  for img in soup.find_all("img", {"src": re.compile(".*\.(jpg|jpeg|png)")}):
      image_url = img.get("src")
      x = image_url[:4]
      # print(x)
      if image_url and x == 'http':
          image_urls.append(image_url)

  # print(image_urls)

  # svg_urls = []
  # for svg in soup.find_all("svg"):
  #     svg_url = svg.get("src")
  #     if svg_url:
  #         svg_urls.append(svg_url)

  # print(svg_urls)

#   gif_urls = []
#   for gif in soup.find_all("img", {"src": re.compile(".*\.gif")}):
#       gif_url = gif.get("src")
#       if gif_url:
#           gif_urls.append(gif_url)

#   print(gif_urls)

  # print(image_urls)
  return image_urls

img_url = web_scrap(url)
# print(len(image_urls))
print(img_url)

['https://assets.shopkund.com/media/catalog/product/cache/3/image/9df78eab33525d08d6e5fb8d27136e95/a/c/act5532-1-weaving-rich-pallu-heavy-embroidery-border-blouse-work-dola-silk-pink-south-indian-saree-blouse-sr19613.jpg', 'https://www.curryflow.com/wp-content/uploads/2021/08/aymen-1.jpg', 'http://ekogate.club/images/sexypussypic.com/galleries/7/374/0_422.jpg', 'http://ekogate.club/images/sexypussypic.com/galleries/10/923/0_876.jpg', 'https://www.curryflow.com/wp-content/uploads/2021/08/hania-3.jpg', 'https://cdn77.scoreuniverse.com/scoreland/gallys/images_content/SigalAcon_35377/07.jpg', 'https://indianxphoto.b-cdn.net/wp-content/uploads/2022/10/20221022_141024.jpg', 'https://static-fhg.metart.com/media/6374C0FB1A089D841D6F8657D85E1727/w_CD1CB6E639805E3429722353385CD5FD.jpg', 'https://t.auntmia.com/nthumbs/2015-10-28/2819342/2819342_04.jpg', 'http://www.desixxxpics.com/wp-content/uploads/2021/12/big-boobs-Indian-bitch.jpg']


### 4. IMAGE PREDICITON

In [16]:
#4. Image prediction

def image_pred(web_url):
  result_url = []
  try:
    list_of_url = web_scrap(web_url)
    # print(list_of_url)
    model = keras.models.load_model('model_94.h5')
    for f in list_of_url:
      im = Image.open(requests.get(f, stream=True).raw)
      img = preprocess_image(im)
      prediction = model.predict(img)
      print("Prediction: ",prediction)
      if prediction[0][0]<=10**-15 and prediction[0][0]!=0.: #threshold
        result_url.append(f)
    return result_url
  except (MissingSchema, InvalidURL):
    pass

x=image_pred(url)
print(x)

1/1 [==============================] - 2s 2s/step
Prediction:  [[5.8590966e-12]]
1/1 [==============================] - 0s 111ms/step
Prediction:  [[0.]]
1/1 [==============================] - 0s 139ms/step
Prediction:  [[0.]]
1/1 [==============================] - 0s 147ms/step
Prediction:  [[8.080287e-31]]
1/1 [==============================] - 0s 134ms/step
Prediction:  [[7.108124e-16]]
1/1 [==============================] - 0s 169ms/step
Prediction:  [[9.778669e-14]]
1/1 [==============================] - 0s 118ms/step
Prediction:  [[0.]]
1/1 [==============================] - 0s 141ms/step
Prediction:  [[2.0616134e-13]]
1/1 [==============================] - 0s 141ms/step
Prediction:  [[0.3217159]]
1/1 [==============================] - 0s 130ms/step
Prediction:  [[3.7216847e-07]]
['http://ekogate.club/images/sexypussypic.com/galleries/10/923/0_876.jpg', 'https://www.curryflow.com/wp-content/uploads/2021/08/hania-3.jpg']


### 5. VIDEO PREDICTION

In [21]:
# #5. Video prediction

# def video_pred(vid_path):
#   vid = cv2.VideoCapture(vid_path)
#   if not os.path.exists('images'):
#     os.makedirs('images')

# #for frame identity
#   index = 0
#   while(True):
#       # Extract images
#       ret, frame = vid.read()
#       # end of frames
#       if not ret: 
#           break
#       # Saves images
#       name = './images/frame' + str(index) + '.jpg'
#       print ('Creating...' + name)
#       cv2.imwrite(name, frame)

#       # next frame
#       index += 1

#   dir_path = r'/content/images/'
#   res=[]

#   for path in os.listdir(dir_path):
#       # check if current path is a file
#       if os.path.isfile(os.path.join(dir_path, path)):
#           res.append(dir_path+path)
#   #print(res)

#   for i in res:
#     img = preprocess_image(i)
#   # Use the ResNet50 model to make a prediction
#     resnet_prediction = model.predict(img)
#   # Print the predictions
#     print('Prediction:', resnet_prediction)

### 6. REPORTING

In [43]:
import os
from email.message import EmailMessage
import ssl
import smtplib
email_sender = 'test9455321@gmail.com'
email_password = 'svpwgwwqknahhrnn'
# email_receiver = input()
email_receiver = data['email']
subject = 'Obscene content found on one of your websites'
body = """Dear Managing Authority,

We are writing to bring to your attention an issue with one of your websites, which we have discovered contains obscene content. As a responsible and reputable website management authority, we trust that you will take prompt action to address this matter.

The website in question is [website URL], which we found to have content that is offensive and inappropriate. This kind of content is not only unacceptable, but it is also a violation of laws and regulations that govern online content.

We understand that managing websites can be challenging, and sometimes such issues may go unnoticed. However, we urge you to take immediate action to remove the obscene content from the website and take necessary measures to ensure that it does not happen again.
We would like to remind you that the presence of such content on your website can have serious consequences, including damage to your reputation and legal repercussions. Therefore, we strongly advise you to take this matter seriously and address it promptly.
We appreciate your prompt attention to this matter, and we hope that you will take appropriate action to ensure that your websites remain free from any inappropriate content.

Thank you for your understanding and cooperation.

Sincerely,
Team Byte Builders
"""
em = EmailMessage()
em['From'] = email_sender
em['To'] = email_receiver
em['Subject'] = subject
em.set_content(body)
context = ssl.create_default_context()
with smtplib.SMTP_SSL('smtp.gmail.com',465,context = context) as smtp:
    smtp.login(email_sender,email_password)
    smtp.sendmail(email_sender,email_receiver,em.as_string())

In [22]:
# def send_mail(send_to_gmail,subject,message,url):
#     gmail= "gmail id"
#     password= "password"
    
#     send_text = image_pred(url)
#     # storing  the information to the server
#     msg=MIMEText(send_text)
#     msg['from']=gmail
#     msg['to']=send_to_gmail
#     msg['subject']= "Obscene Media Detected in the following URLs"

#     # To connect the server
#     server= smtplib.SMTP_SSL('smtp.gmail.com',465)
#     #To login to the server
#     server.login(gmail,password)
#     # Sending the mail
#     server.sendmail(gmail,send_to_gmail,(msg.as_string()))
#     print('Mail sent')
#     # Exiting the server
#     server.quit()

In [46]:
pip install twilio



[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 1.7/1.7 MB 87.2 kB/s eta 0:00:00
     ------------------------------------ 319.8/319.8 kB 154.8 kB/s eta 0:00:00
     -------------------------------------- 60.9/60.9 kB 111.8 kB/s eta 0:00:00


In [47]:
import requests

api_key = 'NjI1NTZkMzI3OTMwNDI2NTZiNjc3ODQ5NjMzNDYyNDk='
sender = 'Divik'
message = 'Hello, this is a test message.'
numbers = '917355192163' # replace with the recipient's phone number

url = 'https://api.textlocal.in/send/'
payload = {
    'apikey': api_key,
    'sender': sender,
    'message': message,
    'numbers': numbers
}


In [ ]:
from firebase_admin import auth

phone_number = '+919149171001' # replace with your phone number

session = auth.create_phone_auth_session(phone_number)
session_id = session.session_info['session_id']


In [ ]:
response = requests.post(url, data=payload)
print(response.text)